<a href="https://colab.research.google.com/github/Nahomie07/students/blob/main/TP_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP INF4137: Implémentation de la méthode SVD pour un systeme de recommandation
NONO NGANSOP NAHOMIE MADELEINE 20U2670

In [ ]:
#Importation des bibliothèques
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_squared_error, precision_score, mean_absolute_error
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [ ]:
#Importation du jeu de donnée
from google.colab import files
import zipfile

# Chargement du dossier
uploaded = files.upload()# nom du fichier zip
archive_name = list(uploaded.keys())[0]

# Extraction des images
with zipfile.ZipFile(archive_name, 'r') as zip_ref:
    zip_ref.extractall()

print("Données extraite avec succès")

Saving donnee.zip to donnee.zip
Données extraite avec succès


## Description du jeu de donnée

Cet ensemble de données (ml-latest-small) décrit l'activité d'évaluation 5 étoiles et de marquage en texte libre de [MovieLens](http://movielens.org), un service de recommandation de films. Il contient 100836 évaluations et 3683 applications de tags sur 9742 films. Ces données ont été créées par 610 utilisateurs entre le 29 mars 1996 et le 24 septembre 2018. Cet ensemble de données a été généré le 26 septembre 2018.

Les utilisateurs ont été sélectionnés au hasard. Tous les utilisateurs sélectionnés ont évalué au moins 20 films. Aucune information démographique n'est incluse. Chaque utilisateur est représenté par un identifiant et aucune autre information n'est fournie.

Les données sont contenues dans les fichiers `links.csv`, `movies.csv`, `ratings.csv` et `tags.csv`.
Mais dans la suite de notre devoir nous n'utiliserons que les fichiers `movies.csv` et `ratings.csv`; surtout le dernier.
- `movies.csv` contient les identifiants (movieId) des films ainsi que leurs titre(title) et genre(genres) correspondant;
- `ratings.csv` contient sur un ligne l'évaluation d'un film par un utilisateur et a le format suivant : userId,movieId,rating,timestamp. Les lignes de ce fichier sont classées d'abord par userId, puis, au sein de l'utilisateur, par movieId. Les notes sont attribuées sur une échelle de 5 étoiles, avec des incréments d'une demi-étoile (0,5 étoile - 5,0 étoiles). Les horodatages représentent les secondes écoulées depuis minuit en temps universel coordonné (UTC) le 1er janvier 1970.

In [ ]:
movies = pd.read_csv('donnee/movies.csv')
ratings = pd.read_csv('donnee/ratings.csv')

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'])

In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,1970-01-01 00:00:00.964982703
1,1,3,4.0,1970-01-01 00:00:00.964981247
2,1,6,4.0,1970-01-01 00:00:00.964982224
3,1,47,5.0,1970-01-01 00:00:00.964983815
4,1,50,5.0,1970-01-01 00:00:00.964982931
...,...,...,...,...
100831,610,166534,4.0,1970-01-01 00:00:01.493848402
100832,610,168248,5.0,1970-01-01 00:00:01.493850091
100833,610,168250,5.0,1970-01-01 00:00:01.494273047
100834,610,168252,5.0,1970-01-01 00:00:01.493846352


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   userId     100836 non-null  int64         
 1   movieId    100836 non-null  int64         
 2   rating     100836 non-null  float64       
 3   timestamp  100836 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 3.1 MB


In [ ]:
ratings.describe()

,userId,movieId,rating
count,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557
std,182.618491,35530.987199,1.042529
min,1.000000,1.000000,0.500000
25%,177.000000,1199.000000,3.000000
50%,325.000000,2991.000000,3.500000
75%,477.000000,8122.000000,4.000000
max,610.000000,193609.000000,5.000000


## Extraction des 30 000 premiers quadruplets

In [ ]:
ratings = ratings[:30000]
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,1970-01-01 00:00:00.964982703
1,1,3,4.0,1970-01-01 00:00:00.964981247
2,1,6,4.0,1970-01-01 00:00:00.964982224
3,1,47,5.0,1970-01-01 00:00:00.964983815
4,1,50,5.0,1970-01-01 00:00:00.964982931
...,...,...,...,...
29995,208,2353,4.0,1970-01-01 00:00:00.940639375
29996,208,2427,3.0,1970-01-01 00:00:00.940639513
29997,208,2571,5.0,1970-01-01 00:00:00.940638581
29998,208,2581,4.0,1970-01-01 00:00:00.940639121


## Partitionnement du jeu de donnée selon la ligne temporelle

On triera les quadruplets extraits dans l’ordre chronologique (croissant suivant le timestamp). Ensuite on divisera le jeu de donnée en deux partie de 21 000(70%) et 30% d'exemples. Et pour le faire, nous sélectionnerons un enregistrement à partir duquel nous diviserons les données. Cet enregistrement divise les données en 70-30%.

In [ ]:
rating = ratings["timestamp"].quantile(0.7)
rating

Timestamp('1970-01-01 00:00:01.377309129')

In [ ]:
train = ratings.loc[ratings['timestamp']<=rating]
test = ratings.loc[ratings['timestamp']>rating]

In [ ]:
train

,userId,movieId,rating,timestamp
0,1,1,4.0,1970-01-01 00:00:00.964982703
1,1,3,4.0,1970-01-01 00:00:00.964981247
2,1,6,4.0,1970-01-01 00:00:00.964982224
3,1,47,5.0,1970-01-01 00:00:00.964983815
4,1,50,5.0,1970-01-01 00:00:00.964982931
...,...,...,...,...
29995,208,2353,4.0,1970-01-01 00:00:00.940639375
29996,208,2427,3.0,1970-01-01 00:00:00.940639513
29997,208,2571,5.0,1970-01-01 00:00:00.940638581
29998,208,2581,4.0,1970-01-01 00:00:00.940639121


In [ ]:
test

,userId,movieId,rating,timestamp
232,2,318,3.0,1970-01-01 00:00:01.445714835
233,2,333,4.0,1970-01-01 00:00:01.445715029
234,2,1704,4.5,1970-01-01 00:00:01.445715228
235,2,3578,4.0,1970-01-01 00:00:01.445714885
236,2,6874,4.0,1970-01-01 00:00:01.445714952
...,...,...,...,...
29931,205,95167,4.0,1970-01-01 00:00:01.519901669
29932,205,96079,3.0,1970-01-01 00:00:01.519900258
29933,205,99114,4.5,1970-01-01 00:00:01.519902005
29934,205,104913,3.0,1970-01-01 00:00:01.519901449


## Implémentation de quelques fonctions utiles

In [ ]:
def sigma(y,U,V):
  """
  Fonction qui crée la matrice diagonale les valeurs sur la diagonale sont les racines carrées des valeurs propres des
  matrices A · AT et AT · A
  Argument:
    y : un np.array unidimentionnel contenant les valeurs propres
    U : matrice des vecteurs propres de droite
    V : matrice des vecteurs propres de gauche
  """
  n = V.shape[1]
  if U.shape[1]< V.shape[1]:
    n = U.shape[1]
  S = np.zeros((n,n))
  for i in range(n):
    S[i][i]=y[i]**(1/2)
  return S

In [ ]:
def HitRate(topNPredicted, leftOutPredictions):
  """
  Fonction qui calcule le hit ratio
  """
  hits = 0
  total = 0

 # For each left-out rating
  for leftOut in leftOutPredictions:
      userID = leftOut[0]
      leftOutMovieID = leftOut[1]
      # Is it in the predicted top 10 for this user?
      hit = False
      for movieID, predictedRating in topNPredicted[int(userID)]:
          if (int(leftOutMovieID) == int(movieID)):
              hit = True
              break
      if (hit) :
          hits += 1

      total += 1

  # Compute overall precision
  return hits/total

## Implementation du SVD

In [ ]:
def mySVD(y,u,v,k):
  """
  Fonction qui effectue SVD
  Argument:
    y : un np.array unidimentionnel contenant les valeurs propres
    u : matrice des vecteurs propres de droite
    v : matrice des vecteurs propres de gauche
  """
  yy = y[:k]
  U = u[:,:k]
  V = v[:,:k]
  S = sigma(yy,U,V)
  return U, S, V

## Application du SVD



### Matrice à décomposer

In [ ]:
A = train.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)
A

movieId,1,2,3,4,5,6,7,8,9,10,...,99114,100383,101864,102125,102445,102481,102716,103042,103801,103819
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,4.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Décomposition

In [ ]:
#Application du SVD
At=A.T
AU = A.dot(At)
AV = At.dot(A)
AU

userId,1,3,4,5,6,7,8,9,11,12,...,197,198,199,200,201,202,204,206,207,208
userId,,,,,,,,,,,,,,,,,,,,,
1,4571.0,80.50,732.0,218.0,552.0,462.00,235.0,103.0,279.0,28.0,...,219.0,1018.0,982.5,910.5,737.0,1633.0,318.00,124.0,111.00,163.0
3,80.5,397.50,2.5,2.5,5.0,0.00,2.5,0.0,0.0,0.0,...,0.0,6.0,5.0,3.5,4.0,84.5,4.75,0.0,0.00,0.0
4,732.0,2.50,3102.0,179.0,314.0,276.00,89.0,15.0,95.0,70.0,...,74.0,724.0,824.0,613.0,376.0,949.0,125.50,87.0,42.50,20.0
5,218.0,2.50,179.0,624.0,478.0,116.50,272.0,0.0,143.0,37.0,...,85.0,232.0,219.0,205.5,136.0,293.0,45.00,80.0,0.00,44.0
6,552.0,5.00,314.0,478.0,4059.0,208.00,599.0,21.0,437.0,154.0,...,99.0,439.0,424.5,513.5,419.0,624.0,82.00,244.0,13.00,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,1633.0,84.50,949.0,293.0,624.0,605.50,278.0,127.0,403.0,92.5,...,391.0,1275.0,1497.5,1043.0,889.0,6095.0,239.50,111.0,90.00,141.0
204,318.0,4.75,125.5,45.0,82.0,149.25,78.5,36.0,22.5,0.0,...,95.5,298.0,426.0,386.0,78.5,239.5,1392.75,0.0,37.50,22.5
206,124.0,0.00,87.0,80.0,244.0,63.00,21.0,0.0,77.0,25.0,...,8.0,130.0,118.0,68.5,89.0,111.0,0.00,425.0,0.00,9.0


In [ ]:
AV

movieId,1,2,3,4,5,6,7,8,9,10,...,99114,100383,101864,102125,102445,102481,102716,103042,103801,103819
movieId,,,,,,,,,,,,,,,,,,,,,
1,922.00,183.75,155.50,0.0,137.50,236.25,164.25,25.0,32.0,238.75,...,0.0,0.0,0.0,0.0,12.5,0.0,0.0,11.25,0.00,0.00
2,183.75,289.75,90.00,12.0,74.00,106.00,75.00,15.0,0.0,153.25,...,0.0,0.0,0.0,0.0,12.5,0.0,0.0,11.25,0.00,0.00
3,155.50,90.00,276.75,15.0,109.00,128.75,165.50,40.0,24.0,89.50,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,9.00,0.00,0.00
4,0.00,12.00,15.00,36.0,24.00,24.00,30.00,9.0,0.0,18.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
5,137.50,74.00,109.00,24.0,251.25,97.00,151.50,40.0,0.0,107.00,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,9.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102481,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,16.0,12.0,0.00,2.00,6.00
102716,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,12.0,9.0,0.00,1.50,4.50
103042,11.25,11.25,9.00,0.0,9.00,18.00,9.00,0.0,0.0,20.25,...,0.0,0.0,0.0,0.0,22.5,0.0,0.0,20.25,0.00,0.00


In [ ]:
yd, u= np.linalg.eigh(AU)
yd = yd[::-1]#Pour ranger en ordre décroissant
u = np.fliplr(u)

In [ ]:
message = "Les {} propres de {} sont:\n {}"
print(message.format("valeurs","droite",yd),"\n")
print(message.format("vecteurs","droite", u))

Les valeurs propres de droite sont:
 [6.12262827e+04 1.34631165e+04 1.12987636e+04 8.64912419e+03
 7.55488797e+03 6.78247544e+03 6.46972518e+03 6.26049960e+03
 5.38801353e+03 5.12059508e+03 4.67621334e+03 4.29835434e+03
 4.02876193e+03 3.90103419e+03 3.75475807e+03 3.60305926e+03
 3.47717616e+03 3.39776364e+03 3.27816569e+03 3.24746285e+03
 3.22576223e+03 3.13830760e+03 3.02141495e+03 2.96876287e+03
 2.83665716e+03 2.68340652e+03 2.62185977e+03 2.58302280e+03
 2.55086843e+03 2.47640381e+03 2.39915221e+03 2.33435430e+03
 2.25875423e+03 2.17393666e+03 2.14681503e+03 2.04472687e+03
 1.99087229e+03 1.96154538e+03 1.94576396e+03 1.86478920e+03
 1.82883345e+03 1.77694570e+03 1.71814373e+03 1.68023712e+03
 1.63502241e+03 1.59270763e+03 1.49251784e+03 1.46074124e+03
 1.41366169e+03 1.39061019e+03 1.38116313e+03 1.33577070e+03
 1.30784862e+03 1.25167087e+03 1.23278341e+03 1.19754273e+03
 1.19501921e+03 1.13223482e+03 1.11908307e+03 1.11275234e+03
 1.06148419e+03 1.03063215e+03 1.01469796e+03 9.

In [ ]:
len(yd)

160

In [ ]:
yg, v= np.linalg.eigh(AV)
yg = yg[::-1]#Pour ranger en ordre décroissant
v = np.fliplr(v)

In [ ]:
message = "Les {} propres de {} sont:\n {}"
print(message.format("valeurs","gauche",yg),"\n")
print(message.format("vecteurs","gauche", v))

Les valeurs propres de gauche sont:
 [ 6.12262827e+04  1.34631165e+04  1.12987636e+04 ... -9.19986002e-12
 -1.15693832e-11 -1.37114639e-11] 

Les vecteurs propres de gauche sont:
 [[ 7.85654459e-02  4.79224607e-02 -2.03966271e-02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 3.33201568e-02  3.24878690e-02  1.75010131e-02 ... -2.20218297e-02
   1.33447475e-01  2.64682423e-01]
 [ 2.88402880e-02  4.88493730e-02  2.77568390e-02 ... -7.87231807e-02
  -4.42139992e-01 -3.84655740e-01]
 ...
 [ 5.68073055e-03 -8.01813984e-03  2.65459179e-02 ...  1.42879613e-02
  -9.15176821e-03 -1.12893718e-02]
 [ 1.55842350e-05 -1.63169196e-05  6.97377729e-05 ... -1.00903083e-03
  -9.46267283e-05  7.17961831e-04]
 [ 4.67527051e-05 -4.89507589e-05  2.09213319e-04 ... -2.91021988e-03
  -3.14881891e-04  2.44768875e-03]]


In [ ]:
len(yg)

4062

In [ ]:
# Application de SVD avec k = 3, 5, 10, 30, 50
UL=[] #Liste contenant les U pour les differents k
SL=[] #pour sigma
VL=[] #pour V
for k in [3, 5, 10, 30, 50]:
  U, S, V = mySVD(yd,u,v,k)
  UL.append(U)
  SL.append(S)
  VL.append(V)

In [ ]:
UL[2]

array([[ 0.13454091,  0.00077013, -0.08547002, ...,  0.03741128,
        -0.14174047,  0.01998762],
       [ 0.00291425,  0.00168386, -0.00257524, ...,  0.00388853,
         0.00417011, -0.00978749],
       [ 0.07915895,  0.02016118, -0.09433397, ..., -0.18832813,
        -0.06394121,  0.03393793],
       ...,
       [ 0.01424086, -0.02687694, -0.00728912, ..., -0.027967  ,
        -0.00989626, -0.01742098],
       [ 0.0050537 ,  0.00449886, -0.00577292, ...,  0.00487058,
        -0.01256083,  0.00259327],
       [ 0.01371039, -0.008744  , -0.00537494, ...,  0.0304803 ,
        -0.01009035,  0.01495353]])

In [ ]:
SL[2]

array([[247.43945256,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        , 116.03067038,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , 106.29564221,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  93.00066767,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         86.91885856,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  82.35578571,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  

In [ ]:
VL[2]

array([[ 7.85654459e-02,  4.79224607e-02, -2.03966271e-02, ...,
         5.55553545e-04, -3.09917190e-02,  3.21948495e-02],
       [ 3.33201568e-02,  3.24878690e-02,  1.75010131e-02, ...,
         2.17688382e-02,  3.97952283e-02,  7.89214288e-02],
       [ 2.88402880e-02,  4.88493730e-02,  2.77568390e-02, ...,
        -2.23041310e-02,  5.84405824e-02, -1.86934933e-03],
       ...,
       [ 5.68073055e-03, -8.01813984e-03,  2.65459179e-02, ...,
        -4.62666034e-03, -1.72035541e-02,  6.96434818e-03],
       [ 1.55842350e-05, -1.63169196e-05,  6.97377729e-05, ...,
         4.62981738e-06, -1.89579608e-06, -5.09872479e-05],
       [ 4.67527051e-05, -4.89507589e-05,  2.09213319e-04, ...,
         1.38894521e-05, -5.68738824e-06, -1.52961744e-04]])

## Calcul des matrices $M_k$ avec $k \in \lbrace 3, 5, 10, 30, 50 \rbrace $

In [ ]:
#Reconstruction de la matrice avec k = 3, 5, 10, 30, 50
M = [] #liste contenant les différentes matrices de reconstruction selon les k
for k in range(5):
  Ap = UL[k].dot(SL[k])
  Ap = Ap.dot(VL[k].T)
  Ap = pd.DataFrame(Ap, index=A.index, columns=A.columns)
  M.append(Ap)


### $M_3$

In [ ]:
M[0]

movieId,1,2,3,4,5,6,7,8,9,10,...,99114,100383,101864,102125,102445,102481,102716,103042,103801,103819
userId,,,,,,,,,,,,,,,,,,,,,
1,2.805088,0.953157,0.712306,0.106449,0.375918,1.963636,0.665173,0.035272,0.021180,1.467887,...,0.036593,-0.002094,-0.001862,-0.001862,-0.058637,-0.000930,-0.000697,-0.052773,-0.000116,-0.000349
3,0.071600,0.025584,0.022743,0.005670,0.014791,0.047579,0.023512,0.003891,0.001341,0.042078,...,0.000940,-0.000815,-0.000724,-0.000724,-0.005263,-0.000088,-0.000066,-0.004737,-0.000011,-0.000033
4,1.855494,0.553155,0.400844,0.103708,0.153817,1.274518,0.415590,0.034990,0.020097,0.907139,...,0.029390,-0.022956,-0.020405,-0.020405,-0.192969,-0.003458,-0.002593,-0.173672,-0.000432,-0.001297
5,0.106316,-0.083659,-0.290208,-0.157098,-0.355363,0.215575,-0.386317,-0.174255,-0.044542,-0.260882,...,0.003952,0.034173,0.030376,0.030376,0.140626,0.002303,0.001727,0.126563,0.000288,0.000864
6,-0.284223,-0.387161,-1.043738,-0.608036,-1.150786,0.317029,-1.428300,-0.617417,-0.165288,-1.127723,...,-0.002788,0.148950,0.132400,0.132400,0.722766,0.012307,0.009230,0.650489,0.001538,0.004615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,3.782999,0.883718,0.401612,0.118712,-0.192819,2.687946,0.422741,-0.071953,0.009247,1.445640,...,0.071200,-0.051955,-0.046182,-0.046182,-0.525085,-0.009752,-0.007314,-0.472576,-0.001219,-0.003657
204,0.946265,0.473642,0.512906,0.107574,0.456954,0.594996,0.513366,0.113832,0.030490,0.761058,...,0.005671,-0.001909,-0.001697,-0.001697,0.037378,0.000931,0.000698,0.033640,0.000116,0.000349
206,0.143200,0.002537,-0.072219,-0.042766,-0.108631,0.142950,-0.100176,-0.053207,-0.012814,-0.032772,...,0.003211,0.008678,0.007714,0.007714,0.027172,0.000414,0.000311,0.024455,0.000052,0.000155


### $M_5$

In [ ]:
M[1]

movieId,1,2,3,4,5,6,7,8,9,10,...,99114,100383,101864,102125,102445,102481,102716,103042,103801,103819
userId,,,,,,,,,,,,,,,,,,,,,
1,3.368385,1.438548,0.521728,-0.033796,0.127359,1.675353,0.421864,-0.116991,-0.006658,1.036818,...,0.064524,-0.016668,-0.014816,-0.014816,-0.049716,-0.002366,-0.001775,-0.044745,-0.000296,-0.000887
3,0.092130,0.041100,0.012356,0.000879,0.005817,0.038540,0.013069,-0.002085,0.000254,0.023777,...,0.002309,-0.000876,-0.000779,-0.000779,-0.005131,-0.000196,-0.000147,-0.004618,-0.000024,-0.000073
4,1.972758,0.717724,0.461658,0.065151,0.099604,1.171637,0.410943,0.015770,0.016428,0.893038,...,0.024935,-0.039699,-0.035288,-0.035288,-0.185465,-0.002152,-0.001614,-0.166918,-0.000269,-0.000807
5,-0.016048,-0.190570,-0.251135,-0.126416,-0.301312,0.279191,-0.334529,-0.141468,-0.038544,-0.168993,...,-0.001878,0.037656,0.033472,0.033472,0.138558,0.002578,0.001933,0.124702,0.000322,0.000967
6,-0.661570,-0.989780,-1.354994,-0.473198,-0.973494,0.697391,-1.466252,-0.569917,-0.155927,-1.169330,...,0.023356,0.218593,0.194304,0.194304,0.692123,0.006261,0.004696,0.622911,0.000783,0.002348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,4.273624,1.677606,0.822703,-0.058131,-0.423737,2.186406,0.479593,-0.131677,-0.002577,1.512079,...,0.035531,-0.144740,-0.128658,-0.128658,-0.484322,-0.001630,-0.001223,-0.435890,-0.000204,-0.000611
204,0.763887,0.430980,0.755729,0.136109,0.532980,0.611068,0.675060,0.185619,0.043335,1.036952,...,-0.021882,-0.021900,-0.019466,-0.019466,0.044668,0.004287,0.003215,0.040202,0.000536,0.001608
206,0.116734,-0.021669,-0.065480,-0.035970,-0.096898,0.157439,-0.089759,-0.046328,-0.011553,-0.014186,...,0.002125,0.009665,0.008591,0.008591,0.026628,0.000446,0.000335,0.023965,0.000056,0.000167


### $M_{10}$

In [ ]:
M[2]

movieId,1,2,3,4,5,6,7,8,9,10,...,99114,100383,101864,102125,102445,102481,102716,103042,103801,103819
userId,,,,,,,,,,,,,,,,,,,,,
1,4.197422,1.301344,-0.227141,-0.106486,0.077349,1.652457,-0.130379,-0.096371,0.005315,1.247877,...,0.055269,-0.000489,-0.000435,-0.000435,-0.082425,-0.004650,-0.003487,-0.074182,-0.000581,-0.001744
3,0.059435,0.025428,0.023784,-0.000680,0.025299,0.041421,0.032302,-0.009039,-0.000905,0.001928,...,0.001095,-0.004989,-0.004435,-0.004435,-0.022910,0.000133,0.000100,-0.020619,0.000017,0.000050
4,1.765646,0.277642,0.583045,0.266427,0.040716,0.926197,0.644416,0.049565,0.011939,0.939807,...,0.033512,-0.070928,-0.063047,-0.063047,0.216431,-0.001857,-0.001393,0.194788,-0.000232,-0.000696
5,0.116159,-0.368533,-0.318235,-0.127955,-0.279668,0.162137,-0.393987,-0.148909,-0.037860,-0.189742,...,-0.008041,0.033293,0.029594,0.029594,0.117218,0.002377,0.001782,0.105496,0.000297,0.000891
6,-1.259071,-1.360847,-1.028245,-0.394841,-0.819798,0.616993,-1.083050,-0.635742,-0.171697,-1.391979,...,0.016940,0.164952,0.146624,0.146624,0.698202,0.010204,0.007653,0.628382,0.001276,0.003827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2.948329,1.901853,0.786551,0.062477,-0.468938,3.080000,0.925043,-0.197779,-0.027612,1.704882,...,0.068029,-0.207083,-0.184074,-0.184074,-0.126364,0.006753,0.005065,-0.113728,0.000844,0.002532
204,1.288146,0.385112,0.711975,0.107796,0.716711,0.262372,0.598601,0.206245,0.048785,0.824380,...,-0.042124,-0.029338,-0.026078,-0.026078,-0.176747,0.001902,0.001427,-0.159073,0.000238,0.000713
206,0.019212,-0.209240,-0.044125,-0.008222,-0.101125,0.110614,-0.048784,-0.060773,-0.014322,-0.016466,...,0.001444,-0.000334,-0.000297,-0.000297,0.079968,0.001229,0.000922,0.071971,0.000154,0.000461


### $M_{30}$

In [ ]:
M[3]

movieId,1,2,3,4,5,6,7,8,9,10,...,99114,100383,101864,102125,102445,102481,102716,103042,103801,103819
userId,,,,,,,,,,,,,,,,,,,,,
1,4.289035,1.547750,-0.297872,0.187581,2.100082,0.211151,1.442259,-0.126829,-0.015728,-0.172815,...,0.314217,0.740281,0.658027,0.658027,0.301957,0.052207,0.039155,0.271761,0.006526,0.019578
3,0.159187,-0.046126,0.104654,-0.011355,0.015622,0.037600,0.048832,-0.019398,0.007333,-0.063445,...,0.008801,0.032252,0.028669,0.028669,-0.023168,0.001367,0.001025,-0.020851,0.000171,0.000513
4,1.556720,1.115211,1.187177,0.711676,-0.517682,-0.336467,2.244822,0.128743,-0.002692,1.518494,...,0.203749,0.380390,0.338125,0.338125,0.202174,0.007671,0.005753,0.181957,0.000959,0.002877
5,0.244809,-0.492612,-0.148640,-0.118611,-0.052260,0.036421,-0.104708,-0.113460,-0.008184,-0.025489,...,0.017958,0.044291,0.039370,0.039370,0.065380,0.008399,0.006299,0.058842,0.001050,0.003150
6,-0.578590,-0.665173,-1.113312,-0.515365,-1.963881,-0.121208,-1.903223,-1.128521,-0.184858,-1.496507,...,-0.004341,0.199319,0.177172,0.177172,0.886921,0.015826,0.011870,0.798229,0.001978,0.005935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2.925119,0.296799,0.288748,-0.601308,-1.588245,2.434277,1.004168,-0.299445,0.151112,0.477949,...,0.093696,-0.391855,-0.348315,-0.348315,0.118072,-0.007654,-0.005741,0.106265,-0.000957,-0.002870
204,1.135127,0.398498,0.956032,0.241680,1.182026,0.381083,1.228217,0.242718,0.066603,0.519040,...,-0.074059,-0.104712,-0.093077,-0.093077,-0.153778,-0.007580,-0.005685,-0.138400,-0.000948,-0.002843
206,-0.341016,0.141446,0.010467,-0.017323,-0.259796,-0.279030,-0.315885,-0.033774,-0.040928,0.268402,...,0.028207,-0.036703,-0.032625,-0.032625,0.020268,0.009662,0.007247,0.018241,0.001208,0.003623


### $M_{50}$

In [ ]:
M[4]

movieId,1,2,3,4,5,6,7,8,9,10,...,99114,100383,101864,102125,102445,102481,102716,103042,103801,103819
userId,,,,,,,,,,,,,,,,,,,,,
1,5.063773,1.394072,0.271482,0.174216,2.595581,0.434448,2.063427,0.229729,0.072708,-0.417000,...,0.300054,0.863065,0.767168,0.767168,0.270116,0.064931,0.048698,0.243104,0.008116,0.024349
3,0.191541,-0.059178,0.221671,-0.018457,0.107035,0.034417,0.170179,0.047090,0.024848,-0.084116,...,0.009464,0.053925,0.047933,0.047933,-0.025159,0.003029,0.002272,-0.022643,0.000379,0.001136
4,5.426922,1.239336,1.918291,0.425259,-1.008832,-0.315781,2.310601,-0.090280,0.342261,1.273540,...,0.307954,-0.357832,-0.318073,-0.318073,0.272738,0.005077,0.003807,0.245464,0.000635,0.001904
5,0.024661,-0.472789,-0.166057,-0.107186,-0.131892,-0.069417,-0.159554,-0.062876,-0.008923,-0.165804,...,0.061861,-0.121536,-0.108032,-0.108032,0.097475,0.016136,0.012102,0.087727,0.002017,0.006051
6,-0.049186,-0.734786,-1.572234,-0.226301,-2.181730,0.051453,-2.684295,-1.580394,-0.368164,-0.607437,...,-0.140570,0.325973,0.289754,0.289754,0.779943,-0.014683,-0.011012,0.701949,-0.001835,-0.005506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2.887330,0.043535,0.199236,-0.615569,-1.635831,2.079063,0.753558,-0.401750,0.108934,-0.077210,...,0.125759,-0.132676,-0.117934,-0.117934,0.059170,-0.005657,-0.004243,0.053253,-0.000707,-0.002121
204,1.985970,1.287495,0.776553,0.241287,1.265300,0.503829,1.418542,0.215564,-0.000761,0.235881,...,-0.149865,0.137099,0.121866,0.121866,-0.181023,0.004774,0.003580,-0.162920,0.000597,0.001790
206,-1.749133,-0.125170,-0.196874,0.073822,-0.332139,-0.395111,-0.516675,-0.114734,-0.148379,0.373801,...,0.022229,0.093297,0.082931,0.082931,-0.013012,0.021148,0.015861,-0.011711,0.002643,0.007930


## Prédiction de note et evaluation

Nous ignorons ceux dont le userId et/ou le movieId est
absent ou sont absent dans le jeu d’entrainement

In [ ]:
#Sélection dans le jeu de test des users présents dans le jeu d'entrainement
tt = test.loc[test['userId'].isin(train['userId'])]
tt

,userId,movieId,rating,timestamp
1434,15,1,2.5,1970-01-01 00:00:01.510577970
1436,15,47,3.5,1970-01-01 00:00:01.510571970
1440,15,260,5.0,1970-01-01 00:00:01.510571946
1441,15,293,3.0,1970-01-01 00:00:01.510571962
1442,15,296,4.0,1970-01-01 00:00:01.510571877
...,...,...,...,...
28654,199,1179,4.5,1970-01-01 00:00:01.401308834
28802,199,3160,4.0,1970-01-01 00:00:01.401308949
28817,199,3868,3.5,1970-01-01 00:00:01.401308851
28883,199,7063,4.0,1970-01-01 00:00:01.401308873


In [ ]:
#Sélection dans le jeu de test des movies présents dans le jeu d'entrainement
t = tt.loc[tt['movieId'].isin(train['movieId'])]
t # Jeu de test final

,userId,movieId,rating,timestamp
1434,15,1,2.5,1970-01-01 00:00:01.510577970
1436,15,47,3.5,1970-01-01 00:00:01.510571970
1440,15,260,5.0,1970-01-01 00:00:01.510571946
1441,15,293,3.0,1970-01-01 00:00:01.510571962
1442,15,296,4.0,1970-01-01 00:00:01.510571877
...,...,...,...,...
28654,199,1179,4.5,1970-01-01 00:00:01.401308834
28802,199,3160,4.0,1970-01-01 00:00:01.401308949
28817,199,3868,3.5,1970-01-01 00:00:01.401308851
28883,199,7063,4.0,1970-01-01 00:00:01.401308873


In [ ]:
# Fonction qui récupère les éléments de la matrice reconstruite qui sont dans le jeu de test
#ou du jeu de test dans la matrice reconstruite
#Voici la fonction qui fera ce travail
def as_test(M,T):
  m = M.merge(T, on=['userId', 'movieId'], how='inner', indicator=True)
  m = m[m['_merge'] == 'both']
  m = m[['userId', 'movieId', 'rating_y','rating_x']
  m = m.rename(columns={'rating_y': 'rating'})
  m = m.rename(columns={'rating_x': 'rating_predict'})
  return m


### Pour k = 3

#### RMSE

In [ ]:
M3 = pd.melt(M[0].reset_index(), id_vars='userId', value_vars= M[0].columns, var_name='movieId', value_name='rating')
M3

,userId,movieId,rating
0,1,1,2.805088
1,3,1,0.071600
2,4,1,1.855494
3,5,1,0.106316
4,6,1,-0.284223
...,...,...,...
649915,202,103819,-0.003657
649916,204,103819,0.000349
649917,206,103819,0.000155
649918,207,103819,-0.000095


In [ ]:
M3= as_test(M3,t)
M3 = M3.sort_values(['userId','movieId'],ascending=True)
M3

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.064532
6,15,47,3.5,0.081997
8,15,260,5.0,0.064257
10,15,293,3.0,0.046813
12,15,296,4.0,0.085380
...,...,...,...,...
46,199,1179,4.5,0.829097
135,199,3160,4.0,1.068318
154,199,3868,3.5,0.570604
225,199,7063,4.0,0.184446


In [ ]:
#RMSE
MSE = mean_squared_error(M3['rating'], M3['rating_predict'])
RMSE = math.sqrt(MSE)
print("Root Mean Square Error for k = 3 :", RMSE)

Root Mean Square Error for k = 3 : 3.548246478728008


Comme les notes varies de 0 à 5, le taux d'erreur ici est environ de 70,92%, qui est élevé

#### Les 10 couples user_movie
la valeur du rating et la valeur
prédite pour laquelle l’erreur de prédiction est minimale.

In [ ]:
# Fonction qui affichera ces couples
def mpred(M):
  M['erreur'] = M['rating']
  l = []
  for i in range(len(M['erreur'])):
    M['erreur'][i] = math.sqrt((M['rating'][i]-M['rating_predict'][i])**2)
  M = M.sort_values('erreur' ,ascending=True)
  return M

In [ ]:
Pr3 = mpred(M3)
Pr3[:10]

<ipython-input-249-e71280085a6b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['erreur'][i] = math.sqrt((M['rating'][i]-M['rating_predict'][i])**2)


,userId,movieId,rating,rating_predict,erreur
25,21,551,0.5,0.082715,0.417285
69,21,1391,0.5,0.060261,0.439739
114,21,2762,0.5,0.058016,0.441984
323,21,66934,0.5,0.054613,0.445387
130,21,3000,0.5,0.018697,0.481303
97,21,2288,0.5,0.013581,0.486419
20,21,435,1.0,0.112342,0.887658
159,21,3988,1.0,0.061060,0.938940
351,21,81591,1.0,0.044388,0.955612
95,21,2174,1.0,0.039596,0.960404


### Pour k = 5

In [ ]:
M5 = pd.melt(M[1].reset_index(), id_vars='userId', value_vars= M[1].columns, var_name='movieId', value_name='rating')
M5

,userId,movieId,rating
0,1,1,3.368385
1,3,1,0.092130
2,4,1,1.972758
3,5,1,-0.016048
4,6,1,-0.661570
...,...,...,...
51835,202,102125,-0.128658
51836,204,102125,-0.019466
51837,206,102125,0.008591
51838,207,102125,-0.001478


In [ ]:
M5= as_test(M5,t)
M5 = M5.sort_values(['userId','movieId'],ascending=True)
M5

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.080673
6,15,47,3.5,0.072825
8,15,260,5.0,0.117538
10,15,293,3.0,0.048748
12,15,296,4.0,0.098167
...,...,...,...,...
46,199,1179,4.5,0.405810
135,199,3160,4.0,1.334038
154,199,3868,3.5,1.075652
225,199,7063,4.0,0.060432


In [ ]:
#RMSE
MSE = mean_squared_error(M5['rating'], M5['rating_predict'])
RMSE = math.sqrt(MSE)
print("Root Mean Square Error for k = 5 :", RMSE)

Root Mean Square Error for k = 5 : 3.5463038719868347


Le taux d'erreur ici est légèrement inférieur au précédent

#### Les 10 couples user_movie
la valeur du rating et la valeur
prédite pour laquelle l’erreur de prédiction est minimale.

In [ ]:
Pr5 = mpred(M5)
Pr5[:10]

<ipython-input-249-e71280085a6b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['erreur'][i] = math.sqrt((M['rating'][i]-M['rating_predict'][i])**2)


,userId,movieId,rating,rating_predict,erreur
114,21,2762,0.5,0.073468,0.426532
323,21,66934,0.5,0.052480,0.447520
25,21,551,0.5,0.051700,0.448300
97,21,2288,0.5,0.038130,0.461870
69,21,1391,0.5,0.005768,0.494232
130,21,3000,0.5,0.001585,0.498415
20,21,435,1.0,0.086499,0.913501
159,21,3988,1.0,0.056375,0.943625
351,21,81591,1.0,0.051481,0.948519
191,21,5266,1.0,0.025656,0.974344


### Pour k = 10

In [ ]:
M10 = pd.melt(M[2].reset_index(), id_vars='userId', value_vars= M[2].columns, var_name='movieId', value_name='rating')
M10

,userId,movieId,rating
0,1,1,4.197422
1,3,1,0.059435
2,4,1,1.765646
3,5,1,0.116159
4,6,1,-1.259071
...,...,...,...
649915,202,103819,0.002532
649916,204,103819,0.000713
649917,206,103819,0.000461
649918,207,103819,-0.000212


In [ ]:
M10= as_test(M10,t)
M10 = M10.sort_values(['userId','movieId'],ascending=True)
M10

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.037176
6,15,47,3.5,0.008844
8,15,260,5.0,0.096175
10,15,293,3.0,0.048667
12,15,296,4.0,0.033717
...,...,...,...,...
46,199,1179,4.5,1.004920
135,199,3160,4.0,1.564070
154,199,3868,3.5,1.161379
225,199,7063,4.0,0.259904


In [ ]:
#RMSE
MSE = mean_squared_error(M10['rating'], M10['rating_predict'])
RMSE = math.sqrt(MSE)
print("Root Mean Square Error for k = 10 :", RMSE)

Root Mean Square Error for k = 10 : 3.549234205119997


Le taux d'erreur ici est inférieur aux autre

#### Les 10 couples user_movie
la valeur du rating et la valeur
prédite pour laquelle l’erreur de prédiction est minimale.

In [ ]:
Pr10 = mpred(M10)
Pr10[:10]

<ipython-input-249-e71280085a6b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['erreur'][i] = math.sqrt((M['rating'][i]-M['rating_predict'][i])**2)


,userId,movieId,rating,rating_predict,erreur
323,21,66934,0.5,0.065255,0.434745
25,21,551,0.5,0.060572,0.439428
97,21,2288,0.5,0.051543,0.448457
114,21,2762,0.5,0.039724,0.460276
69,21,1391,0.5,0.019661,0.480339
130,21,3000,0.5,0.010848,0.489152
20,21,435,1.0,0.080579,0.919421
159,21,3988,1.0,0.077877,0.922123
351,21,81591,1.0,0.057329,0.942671
191,21,5266,1.0,0.032764,0.967236


### Pour k = 30

In [ ]:
M30 = pd.melt(M[3].reset_index(), id_vars='userId', value_vars= M[3].columns, var_name='movieId', value_name='rating')
M30

,userId,movieId,rating
0,1,1,4.289035
1,3,1,0.159187
2,4,1,1.556720
3,5,1,0.244809
4,6,1,-0.578590
...,...,...,...
649915,202,103819,-0.002870
649916,204,103819,-0.002843
649917,206,103819,0.003623
649918,207,103819,-0.000222


In [ ]:
M30= as_test(M30,t)
M30 = M30.sort_values(['userId','movieId'],ascending=True)
M30

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.115474
6,15,47,3.5,-0.000993
8,15,260,5.0,0.153700
10,15,293,3.0,0.069672
12,15,296,4.0,0.020829
...,...,...,...,...
46,199,1179,4.5,2.273234
135,199,3160,4.0,2.237969
154,199,3868,3.5,0.196389
225,199,7063,4.0,0.105384


In [ ]:
#RMSE
MSE = mean_squared_error(M30['rating'], M30['rating_predict'])
RMSE = math.sqrt(MSE)
print("Root Mean Square Error for k = 30 :", RMSE)

Root Mean Square Error for k = 30 : 3.5447406064332094


Le taux d'erreur diminue légèrement

#### Les 10 couples user_movie
la valeur du rating et la valeur
prédite pour laquelle l’erreur de prédiction est minimale.

In [ ]:
Pr30 = mpred(M30)
Pr30[:10]

<ipython-input-249-e71280085a6b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['erreur'][i] = math.sqrt((M['rating'][i]-M['rating_predict'][i])**2)


,userId,movieId,rating,rating_predict,erreur
114,21,2762,0.5,0.069782,0.430218
323,21,66934,0.5,0.063960,0.436040
69,21,1391,0.5,0.012496,0.487504
25,21,551,0.5,0.008210,0.491790
43,199,969,4.0,3.475976,0.524024
130,21,3000,0.5,-0.026432,0.526432
97,21,2288,0.5,-0.087226,0.587226
191,21,5266,1.0,0.081029,0.918971
95,21,2174,1.0,0.059631,0.940369
20,21,435,1.0,0.047991,0.952009


### Pour k = 50

In [ ]:
M50 = pd.melt(M[4].reset_index(), id_vars='userId', value_vars= M[4].columns, var_name='movieId', value_name='rating')
M50

,userId,movieId,rating
0,1,1,5.063773
1,3,1,0.191541
2,4,1,5.426922
3,5,1,0.024661
4,6,1,-0.049186
...,...,...,...
649915,202,103819,-0.002121
649916,204,103819,0.001790
649917,206,103819,0.007930
649918,207,103819,0.000041


In [ ]:
M50= as_test(M50,t)
M50 = M50.sort_values(['userId','movieId'],ascending=True)
M50

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.075709
6,15,47,3.5,0.036807
8,15,260,5.0,0.149522
10,15,293,3.0,0.050447
12,15,296,4.0,0.119010
...,...,...,...,...
46,199,1179,4.5,2.202567
135,199,3160,4.0,2.188949
154,199,3868,3.5,0.064930
225,199,7063,4.0,0.208848


In [ ]:
#RMSE
MSE = mean_squared_error(M50['rating'], M50['rating_predict'])
RMSE = math.sqrt(MSE)
print("Root Mean Square Error for k = 50 :", RMSE)

Root Mean Square Error for k = 50 : 3.5720680965254243


Le taux d'erreur ici est environne toujours les 70% ce qui est toujours élevé

#### Les 10 couples user_movie
la valeur du rating et la valeur
prédite pour laquelle l’erreur de prédiction est minimale.

In [ ]:
Pr50 = mpred(M50)
Pr50[:10]

<ipython-input-249-e71280085a6b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['erreur'][i] = math.sqrt((M['rating'][i]-M['rating_predict'][i])**2)


,userId,movieId,rating,rating_predict,erreur
25,21,551,0.5,0.122707,0.377293
69,21,1391,0.5,0.121688,0.378312
323,21,66934,0.5,0.074012,0.425988
114,21,2762,0.5,-0.000719,0.500719
97,21,2288,0.5,-0.064176,0.564176
130,21,3000,0.5,-0.158500,0.658500
43,199,969,4.0,3.317322,0.682678
95,21,2174,1.0,0.099788,0.900212
191,21,5266,1.0,0.093946,0.906054
20,21,435,1.0,0.048200,0.951800


## Top-N et Evaluation

Dans la suite, nous ignorons également les couples
userId_movieId présent dans le jeu d’entrainement

In [ ]:
#Voici la fonction qui fera ce travail
def u_m(M):
  m = M.merge(train, on=['userId', 'movieId'], how='left', indicator=True)
  m = m[m['_merge'] == 'left_only']
  m = m[['userId', 'movieId', 'rating_x', 'rating_predict']]
  m = m.rename(columns={'rating_x': 'rating'})
  return m


In [ ]:
#fonction qui fera la recommandation top -N
def top(P, N):
  l = P.index
  p = P.T
  tp = pd.DataFrame()
  A = []
  col = []
  for i in l:
    t = pd.DataFrame(p[i], index=p[i].index)
    t = t.sort_values(t.columns[0], ascending=False)
    a = []
    for j in range(len(t)):
      a.append(t.index[j])
    A.append(a)
    col.append(i)
  A = np.asarray(A)
  A = A.T
  tp = pd.DataFrame(A, columns = col )
  message = "Top {} du classement des films pour les utilisateurs présent dans le jeu de test pour k = {} \n"
  return tp[:N], message

### Pour k = 3

In [ ]:
p3= u_m(M3)
p3

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.064532
1,15,47,3.5,0.081997
2,15,260,5.0,0.064257
3,15,293,3.0,0.046813
4,15,296,4.0,0.085380
...,...,...,...,...
380,199,1179,4.5,0.829097
381,199,3160,4.0,1.068318
382,199,3868,3.5,0.570604
383,199,7063,4.0,0.184446


In [ ]:
r3 = p3.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)
r3

<ipython-input-278-348fae8e1d6d>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  r3 = p3.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,0.064532,0.000000,0.000000,0.000000,0.0000,0.081997,0.000000,0.064257,0.046813,0.085380,...,0.011295,0.000000,0.000000,0.000000,0.000000,0.00000,0.00027,-0.000087,0.001475,0.000000
21,0.138841,0.105439,0.154184,0.000000,0.0911,0.000000,0.199908,0.115963,0.102316,0.168986,...,0.000000,0.032768,0.000000,0.047314,0.054902,0.04369,0.00000,-0.001255,0.005112,0.005112
29,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
68,0.000000,0.000000,0.000000,-0.404957,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,-0.054709,0.000000,0.000000
80,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
104,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.341489,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
199,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000


In [ ]:
P3, mes = top(r3, 10)
print(mes.format(10,3))
P3

Top 10 du classement des films pour les utilisateurs présent dans le jeu de test pour k = 3 



,15,21,29,68,80,104,199
0,356,588,5464,4366,593,4963,3160
1,318,356,1408,1,5630,56367,969
2,588,364,4223,8810,5679,76093,1179
3,364,597,5010,34048,53000,95449,3868
4,296,480,6502,33794,6754,47997,7063
5,47,165,32587,33679,89470,1,32296
6,527,500,34048,33615,42738,33794,33794
7,780,592,33794,33493,3535,33679,33679
8,589,4993,33679,33004,53953,33615,33615
9,2571,4306,33615,32587,8861,33493,33493


### Evaluation

In [ ]:
rr3 = p3.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)
rr3

<ipython-input-280-5cc743a7bbb9>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  rr3 = p3.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,2.5,0.0,0.0,0.0,0.0,3.5,0.0,5.0,3.0,4.0,...,3.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,4.0,0.0
21,3.5,3.5,5.0,0.0,2.5,0.0,3.5,4.5,3.0,3.5,...,0.0,3.5,0.0,3.5,4.0,2.0,0.0,3.5,4.5,3.5
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
PP3, mes = top(rr3, 10)
PP3

,15,21,29,68,80,104,199
0,1200,1270,1408,30749,593,47997,1179
1,2011,10,5464,4366,86190,76093,58998
2,1196,2012,6502,99114,3535,56367,7063
3,1210,2011,4223,18,6754,4963,3160
4,1214,47997,5010,1,8957,95449,969
5,1270,5445,32587,32587,65682,1,3868
6,5989,648,34048,34048,42738,33794,30793
7,589,33004,33794,33794,1200,33679,30816
8,4995,7569,33679,33679,5630,33615,30825
9,4720,1544,33615,33615,1258,33493,30749


In [ ]:
#Précision
def prec(R,P,d = 0):
  precision = []
  m = 0
  p=0
  for i in R.columns:
    if d == 0:
      n = precision_score(R[i], P[i], average='macro', zero_division=1)
    else:
      n = average_precision_score(R[i], P[i])
    precision.append([n])
    m+=n
    p+=1
  m =m/p
  precision = np.asarray(precision)
  #precision = precision.T
  precision= pd.DataFrame(precision, index = R.columns, columns = ["Précision"] )
  message = "La précision pour k = {} est: "
  return m, precision, message

In [ ]:
a3, b3, c3 = prec(PP3, P3)
print(c3.format("3"),a3)

La précision pour k = 3 est:  0.42920763447079235


In [ ]:
b3

,Précision
15,0.473684
21,0.500000
29,0.500000
68,0.230769
80,0.466667
104,0.500000
199,0.333333


In [ ]:
#MAE
mean_absolute_error(PP3, P3)

12347.814285714285

### Les 5 users et leur recommandation Top-10
Pour lesquels ont la
les plus grandes valeurs de PRECISION (Ici on a que 7 users)

In [ ]:
b3 = b3.sort_values("Précision", ascending = False)
b3

,Précision
21,0.500000
29,0.500000
104,0.500000
15,0.473684
80,0.466667
199,0.333333
68,0.230769


In [ ]:
b3 = b3[:5].T

In [ ]:
mP3 = P3.T.loc[P3.columns.isin(b3.columns)]
mP3.T

,15,21,29,80,104
0,356,588,5464,593,4963
1,318,356,1408,5630,56367
2,588,364,4223,5679,76093
3,364,597,5010,53000,95449
4,296,480,6502,6754,47997
5,47,165,32587,89470,1
6,527,500,34048,42738,33794
7,780,592,33794,3535,33679
8,589,4993,33679,53953,33615
9,2571,4306,33615,8861,33493


### Pour k = 5

In [ ]:
p5 = u_m(M5)
p5

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.080673
1,15,47,3.5,0.072825
2,15,260,5.0,0.117538
3,15,293,3.0,0.048748
4,15,296,4.0,0.098167
...,...,...,...,...
380,199,1179,4.5,0.405810
381,199,3160,4.0,1.334038
382,199,3868,3.5,1.075652
383,199,7063,4.0,0.060432


In [ ]:
r5 = p5.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)
r5

<ipython-input-295-affc0f2fed53>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  r5 = p5.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,0.080673,0.000000,0.000000,0.000000,0.000000,0.072825,0.000000,0.117538,0.048748,0.098167,...,0.012755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000024,0.002248,0.002925,0.000000
21,0.157347,0.094095,0.107526,0.000000,0.075776,0.000000,0.160347,0.189187,0.129567,0.242584,...,0.000000,0.031488,0.000000,0.049552,0.058394,0.041984,0.000000,0.004075,0.009922,0.009922
29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
68,0.000000,0.000000,0.000000,-0.567031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.040571,0.000000,0.000000
80,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.341388,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
199,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
P5, mes = top(r5, 10)
print(mes.format(10,5))
P5

Top 10 du classement des films pour les utilisateurs présent dans le jeu de test pour k = 5 



,15,21,29,68,80,104,199
0,356,356,1408,4366,593,4963,3160
1,260,588,5464,1,3535,56367,3868
2,1210,7153,8957,8810,6323,76093,1179
3,1196,296,34150,34048,6754,95449,969
4,2028,480,34048,33794,3826,47997,7063
5,2571,4993,33794,33679,5679,1,32296
6,318,5952,33679,33615,5630,33794,33794
7,1198,780,33615,33493,53000,33679,33679
8,588,364,33493,33004,1258,33615,33615
9,296,6539,33004,32587,53953,33493,33493


#### Evaluation

In [ ]:
rr5 = p5.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)
rr5

<ipython-input-297-e28222901b08>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  rr5 = p5.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,2.5,0.0,0.0,0.0,0.0,3.5,0.0,5.0,3.0,4.0,...,3.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,4.0,0.0
21,3.5,3.5,5.0,0.0,2.5,0.0,3.5,4.5,3.0,3.5,...,0.0,3.5,0.0,3.5,4.0,2.0,0.0,3.5,4.5,3.5
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
PP5, mes = top(rr5, 10)
PP5

,15,21,29,68,80,104,199
0,1200,1270,1408,30749,593,47997,1179
1,2011,10,5464,4366,86190,76093,58998
2,1196,2012,6502,99114,3535,56367,7063
3,1210,2011,4223,18,6754,4963,3160
4,1214,47997,5010,1,8957,95449,969
5,1270,5445,32587,32587,65682,1,3868
6,5989,648,34048,34048,42738,33794,30793
7,589,33004,33794,33794,1200,33679,30816
8,4995,7569,33679,33679,5630,33615,30825
9,4720,1544,33615,33615,1258,33493,30749


In [ ]:
a5, b5, c5 = prec(PP5, P5)
print(c5.format("5"),a5)

La précision pour k = 5 est:  0.414826443397872


In [ ]:
b5

,Précision
15,0.444444
21,0.500000
29,0.428571
68,0.230769
80,0.466667
104,0.500000
199,0.333333


In [ ]:
#MAE
mean_absolute_error(PP5, P5)

14433.600000000002

### Les 5 users et leur recommandation Top-10
Pour lesquels ont la
les plus grandes valeurs de PRECISION (Ici on a que 7 users)

In [ ]:
b5 = b5.sort_values("Précision", ascending = False)
b5

,Précision
21,0.500000
104,0.500000
80,0.466667
15,0.444444
29,0.428571
199,0.333333
68,0.230769


In [ ]:
b5 = b5[:5].T

In [ ]:
mP5 = P5.T.loc[P5.columns.isin(b5.columns)]
mP5.T

,15,21,29,80,104
0,356,356,1408,593,4963
1,260,588,5464,3535,56367
2,1210,7153,8957,6323,76093
3,1196,296,34150,6754,95449
4,2028,480,34048,3826,47997
5,2571,4993,33794,5679,1
6,318,5952,33679,5630,33794
7,1198,780,33615,53000,33679
8,588,364,33493,1258,33615
9,296,6539,33004,53953,33493


### Pour k = 10

In [ ]:
p10 = u_m(M10)
p10

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.037176
1,15,47,3.5,0.008844
2,15,260,5.0,0.096175
3,15,293,3.0,0.048667
4,15,296,4.0,0.033717
...,...,...,...,...
380,199,1179,4.5,1.004920
381,199,3160,4.0,1.564070
382,199,3868,3.5,1.161379
383,199,7063,4.0,0.259904


In [ ]:
r10 = p10.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)
r10

<ipython-input-306-cd04c2cce92c>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  r10 = p10.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,0.037176,0.000000,0.000000,0.000000,0.000000,0.008844,0.000000,0.096175,0.048667,0.033717,...,0.017966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000251,0.003568,0.001707,0.000000
21,0.147777,0.148539,0.112175,0.000000,0.042447,0.000000,0.162568,0.168839,0.135964,0.154189,...,0.000000,0.039153,0.000000,0.057363,0.069429,0.052204,0.000000,0.005600,0.009511,0.009511
29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
68,0.000000,0.000000,0.000000,-0.848621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.037288,0.000000,0.000000
80,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.492225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
199,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
P10, mes = top(r10, 10)
print(mes.format(10,10))
P10

Top 10 du classement des films pour les utilisateurs présent dans le jeu de test pour k = 10 



,15,21,29,68,80,104,199
0,2028,356,1408,4366,593,56367,3160
1,858,588,5464,30749,3535,76093,3868
2,1198,480,4223,1,5630,95449,1179
3,356,7153,8984,32296,6754,47997,969
4,260,364,30749,33794,53322,4963,7063
5,1210,780,34150,33679,5679,1,58998
6,780,5952,34048,33615,3826,33794,30793
7,1196,500,33794,33493,6323,33679,33493
8,5989,5989,33679,33004,1258,33615,34048
9,1270,597,33615,32587,8957,33493,33794


#### Evaluation

In [ ]:
rr10 = p10.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)
rr10

<ipython-input-308-336e5bfe35e1>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  rr10 = p10.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,2.5,0.0,0.0,0.0,0.0,3.5,0.0,5.0,3.0,4.0,...,3.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,4.0,0.0
21,3.5,3.5,5.0,0.0,2.5,0.0,3.5,4.5,3.0,3.5,...,0.0,3.5,0.0,3.5,4.0,2.0,0.0,3.5,4.5,3.5
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
PP10, mes = top(rr10, 10)
PP10

,15,21,29,68,80,104,199
0,1200,1270,1408,30749,593,47997,1179
1,2011,10,5464,4366,86190,76093,58998
2,1196,2012,6502,99114,3535,56367,7063
3,1210,2011,4223,18,6754,4963,3160
4,1214,47997,5010,1,8957,95449,969
5,1270,5445,32587,32587,65682,1,3868
6,5989,648,34048,34048,42738,33794,30793
7,589,33004,33794,33794,1200,33679,30816
8,4995,7569,33679,33679,5630,33615,30825
9,4720,1544,33615,33615,1258,33493,30749


In [ ]:
a10, b10, c10 = prec(PP10, P10)
print(c10.format("10"),a10)

La précision pour k = 10 est:  0.4477629513343799


In [ ]:
b10

,Précision
15,0.375000
21,0.500000
29,0.692308
68,0.230769
80,0.428571
104,0.600000
199,0.307692


In [ ]:
#MAE
mean_absolute_error(PP10, P10)

13318.314285714287

### Les 5 users et leur recommandation Top-10
Pour lesquels ont la
les plus grandes valeurs de PRECISION (Ici on a que 7 users)

In [ ]:
b10 = b10.sort_values("Précision", ascending = False)
b10

,Précision
29,0.692308
104,0.600000
21,0.500000
80,0.428571
15,0.375000
199,0.307692
68,0.230769


In [ ]:
b10 = b10[:5].T

In [ ]:
mP10 = P10.T.loc[P10.columns.isin(b10.columns)]
mP10.T

,15,21,29,80,104
0,2028,356,1408,593,56367
1,858,588,5464,3535,76093
2,1198,480,4223,5630,95449
3,356,7153,8984,6754,47997
4,260,364,30749,53322,4963
5,1210,780,34150,5679,1
6,780,5952,34048,3826,33794
7,1196,500,33794,6323,33679
8,5989,5989,33679,1258,33615
9,1270,597,33615,8957,33493


### Pour k = 30

In [ ]:
p30 = u_m(M30)
p30

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.115474
1,15,47,3.5,-0.000993
2,15,260,5.0,0.153700
3,15,293,3.0,0.069672
4,15,296,4.0,0.020829
...,...,...,...,...
380,199,1179,4.5,2.273234
381,199,3160,4.0,2.237969
382,199,3868,3.5,0.196389
383,199,7063,4.0,0.105384


In [ ]:
r30 = p30.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)
r30

<ipython-input-317-8b2c284d54eb>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  r30 = p30.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,0.115474,0.000000,0.000000,0.000000,0.00000,-0.000993,0.000000,0.153700,0.069672,0.020829,...,0.020884,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001033,0.002469,0.002011,0.000000
21,0.156281,0.108575,-0.088248,0.000000,-0.01124,0.000000,0.006733,0.402037,0.314886,0.156799,...,0.000000,0.038376,0.000000,0.048484,0.038647,0.051168,0.000000,-0.009471,-0.016815,-0.016815
29,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
68,0.000000,0.000000,0.000000,-1.212013,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.037039,0.000000,0.000000
80,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
104,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.560532,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
199,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
P30, mes = top(r30, 10)
print(mes.format(10,30))
P30

Top 10 du classement des films pour les utilisateurs présent dans le jeu de test pour k = 30 



,15,21,29,68,80,104,199
0,2028,260,1408,4366,593,4963,969
1,780,480,4223,30749,6754,76093,1179
2,1198,293,1,1,3826,56367,3160
3,356,780,34162,32296,1258,95449,3868
4,260,356,34048,33794,6323,47997,7063
5,527,1210,33794,33679,53322,1,32296
6,1210,41566,33679,33615,53953,33794,33794
7,858,1196,33615,33493,53000,33679,33679
8,318,1198,33493,33004,89470,33615,33615
9,1200,1580,33004,32587,8861,33493,33493


#### Evaluation

In [ ]:
rr30 = p30.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)
rr30

<ipython-input-319-e25970cd4ae1>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  rr30 = p30.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,2.5,0.0,0.0,0.0,0.0,3.5,0.0,5.0,3.0,4.0,...,3.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,4.0,0.0
21,3.5,3.5,5.0,0.0,2.5,0.0,3.5,4.5,3.0,3.5,...,0.0,3.5,0.0,3.5,4.0,2.0,0.0,3.5,4.5,3.5
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
PP30, mes = top(rr30, 10)
PP30

,15,21,29,68,80,104,199
0,1200,1270,1408,30749,593,47997,1179
1,2011,10,5464,4366,86190,76093,58998
2,1196,2012,6502,99114,3535,56367,7063
3,1210,2011,4223,18,6754,4963,3160
4,1214,47997,5010,1,8957,95449,969
5,1270,5445,32587,32587,65682,1,3868
6,5989,648,34048,34048,42738,33794,30793
7,589,33004,33794,33794,1200,33679,30816
8,4995,7569,33679,33679,5630,33615,30825
9,4720,1544,33615,33615,1258,33493,30749


In [ ]:
a30, b30, c30 = prec(PP30, P30)
print(c30.format("30"),a30)

La précision pour k = 30 est:  0.43375401442628336


In [ ]:
b30

,Précision
15,0.444444
21,0.500000
29,0.357143
68,0.230769
80,0.470588
104,0.700000
199,0.333333


In [ ]:
#MAE
mean_absolute_error(PP30, P30)

14117.9

### Les 5 users et leur recommandation Top-10
Pour lesquels ont la
les plus grandes valeurs de PRECISION (Ici on a que 7 users)

In [ ]:
b30 = b30.sort_values("Précision", ascending = False)
b30

,Précision
104,0.700000
21,0.500000
80,0.470588
15,0.444444
29,0.357143
199,0.333333
68,0.230769


In [ ]:
b30 = b30[:5].T

In [ ]:
mP30 = P30.T.loc[P30.columns.isin(b30.columns)]
mP30.T

,15,21,29,80,104
0,2028,260,1408,593,4963
1,780,480,4223,6754,76093
2,1198,293,1,3826,56367
3,356,780,34162,1258,95449
4,260,356,34048,6323,47997
5,527,1210,33794,53322,1
6,1210,41566,33679,53953,33794
7,858,1196,33615,53000,33679
8,318,1198,33493,89470,33615
9,1200,1580,33004,8861,33493


### Pour k = 50

In [ ]:
p50 = u_m(M50)
p50

,userId,movieId,rating,rating_predict
0,15,1,2.5,0.075709
1,15,47,3.5,0.036807
2,15,260,5.0,0.149522
3,15,293,3.0,0.050447
4,15,296,4.0,0.119010
...,...,...,...,...
380,199,1179,4.5,2.202567
381,199,3160,4.0,2.188949
382,199,3868,3.5,0.064930
383,199,7063,4.0,0.208848


In [ ]:
r50 = p50.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)
r50

<ipython-input-328-806a6fd86cee>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  r50 = p50.pivot(columns = 'movieId', index = 'userId', values = 'rating_predict').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,0.075709,0.0000,0.000000,0.000000,0.000000,0.036807,0.000000,0.149522,0.050447,0.119010,...,0.009079,0.000000,0.000000,0.000000,0.000000,0.00000,-0.003665,-0.009771,-0.010020,0.000000
21,0.519986,-0.0097,-0.346195,0.000000,0.051821,0.000000,-0.075414,0.378499,0.388310,0.270565,...,0.000000,0.044407,0.000000,0.058088,0.018722,0.05921,0.000000,-0.022102,-0.042568,-0.042568
29,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
68,0.000000,0.0000,0.000000,-1.224437,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,-0.020782,0.000000,0.000000
80,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
104,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.568436,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
199,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [ ]:
P50, mes = top(r50, 10)
print(mes.format(10,3))
P50

Top 10 du classement des films pour les utilisateurs présent dans le jeu de test pour k = 3 



,15,21,29,68,80,104,199
0,2028,1,1,4366,593,4963,969
1,588,5952,51084,30749,3826,76093,1179
2,1210,293,34150,1,6754,95449,3160
3,589,260,34048,32296,53322,56367,7063
4,1196,41566,33794,33794,53953,47997,3868
5,260,588,33679,33679,42738,1,32296
6,2571,780,33615,33615,8810,33794,33794
7,1198,541,33493,33493,40583,33679,33679
8,527,5816,33004,33004,65682,33615,33615
9,296,2023,32587,32587,5630,33493,33493


#### Evaluation

In [ ]:
rr50 = p50.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)
rr50

<ipython-input-330-00aa453c6530>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  rr50 = p50.pivot(columns = 'movieId', index = 'userId', values = 'rating').fillna(0)


movieId,1,2,10,18,19,47,165,260,293,296,...,94864,95167,95449,95510,96079,97913,97938,99114,101864,102125
userId,,,,,,,,,,,,,,,,,,,,,
15,2.5,0.0,0.0,0.0,0.0,3.5,0.0,5.0,3.0,4.0,...,3.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,4.0,0.0
21,3.5,3.5,5.0,0.0,2.5,0.0,3.5,4.5,3.0,3.5,...,0.0,3.5,0.0,3.5,4.0,2.0,0.0,3.5,4.5,3.5
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
PP50, mes = top(rr50, 10)
PP50

,15,21,29,68,80,104,199
0,1200,1270,1408,30749,593,47997,1179
1,2011,10,5464,4366,86190,76093,58998
2,1196,2012,6502,99114,3535,56367,7063
3,1210,2011,4223,18,6754,4963,3160
4,1214,47997,5010,1,8957,95449,969
5,1270,5445,32587,32587,65682,1,3868
6,5989,648,34048,34048,42738,33794,30793
7,589,33004,33794,33794,1200,33679,30816
8,4995,7569,33679,33679,5630,33615,30825
9,4720,1544,33615,33615,1258,33493,30749


In [ ]:
a50, b50, c50 = prec(PP50, P50)
print(c50.format("50"),a50)

La précision pour k = 50 est:  0.4013143719026072


In [ ]:
b50

,Précision
15,0.411765
21,0.500000
29,0.333333
68,0.230769
80,0.400000
104,0.600000
199,0.333333


In [ ]:
#MAE
mean_absolute_error(PP50, P50)

15132.557142857144

### Les 5 users et leur recommandation Top-10
Pour lesquels ont la
les plus grandes valeurs de PRECISION (Ici on a que 7 users)

In [ ]:
b50 = b50.sort_values("Précision", ascending = False)
b50

,Précision
104,0.600000
21,0.500000
15,0.411765
80,0.400000
29,0.333333
199,0.333333
68,0.230769


In [ ]:
b50 = b50[:5].T

In [ ]:
mP50 = P50.T.loc[P50.columns.isin(b50.columns)]
mP50.T

,15,21,29,80,104
0,2028,1,1,593,4963
1,588,5952,51084,3826,76093
2,1210,293,34150,6754,95449
3,589,260,34048,53322,56367
4,1196,41566,33794,53953,47997
5,260,588,33679,42738,1
6,2571,780,33615,8810,33794
7,1198,541,33493,40583,33679
8,527,5816,33004,65682,33615
9,296,2023,32587,5630,33493


In [ ]:
# NONO NGANSOP NAHOMIE MADELEINE 20U2670